## Data Cleaning

In [ ]:
import pandas_datareader

In [ ]:
# Import the libs

import math
import pandas_datareader as web #for reading the data
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential #using keras
from keras.layers import Dense, LSTM #importing LSTMs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') #specfic style of the matplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/Shareddrives/IDS561-BigData/Data/SentimentAnalysis

/content/drive/Shareddrives/IDS561-BigData/Data/SentimentAnalysis


In [ ]:
!ls

final_experiment  tesla_tweets_2018_2020.csv


Install Spark 

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -v https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

--2022-04-22 17:22:44--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M  49.4MB/s    in 4.5s    

2022-04-22 17:22:49 (48.5 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [ ]:
!ls

final_experiment  spark-3.1.2-bin-hadoop3.2.tgz  tesla_tweets_2018_2020.csv


#### Untar the Spark installer

In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

#### Check the Spark folder after untar

In [ ]:
!ls 

final_experiment	   spark-3.1.2-bin-hadoop3.2.tgz
spark-3.1.2-bin-hadoop3.2  tesla_tweets_2018_2020.csv


#### Install findspark - a python library to find Spark

In [ ]:
!pip install -q findspark

#### Set environment variables
Set Java and Spark home based on the location where they are stored

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/Shareddrives/IDS561-BigData/Data/SentimentAnalysis/spark-3.1.2-bin-hadoop3.2"

#### Create a local Spark session

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Instantiate SparkNLP Session
#spark = sparknlp.start()
#spark = SparkSession.builder \
#    .appName("Twitter Posts Study") \
#    .config('spark.jars.packages', #need specify this spark session to read in data (and do PySpark df stuff)
#            'com.johnsnowlabs.nlp:spark-nlp_2.11:2.3.5') \
#    .getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

In [ ]:
!ls

final_experiment	   spark-3.1.2-bin-hadoop3.2.tgz
spark-3.1.2-bin-hadoop3.2  tesla_tweets_2018_2020.csv


In [ ]:
df_spark = spark.read.format("csv").option("multiLine", True).option("header", True).option("escape", "\"").load("tesla_tweets_2018_2020.csv") #Loading the csv file into spark

In [ ]:
df_spark.head()

Row(_c0='0', created_at='2020-06-01T22:59:51+00:00', id_str='1267591722126598144', conversation_id_str='1267591722126598144', full_text='I just got hired at Tesla! God is good.', lang='en', favorited='False', retweeted='False', retweet_count='0', favorite_count='19', reply_count='5', quote_count='0', quoted_status_id_str=None, quoted_status_short_url=None, quoted_status_expand_url=None, user_id_str='247611935', user_name='J0rdan0re0', user_full_name='Jordy', user_verified='False', in_reply_to_status_id_str=None, in_reply_to_user_id_str=None, hashtags=None, mentions=None, urls=None, media='[]')

In [ ]:
df_spark.columns

['_c0',
 'created_at',
 'id_str',
 'conversation_id_str',
 'full_text',
 'lang',
 'favorited',
 'retweeted',
 'retweet_count',
 'favorite_count',
 'reply_count',
 'quote_count',
 'quoted_status_id_str',
 'quoted_status_short_url',
 'quoted_status_expand_url',
 'user_id_str',
 'user_name',
 'user_full_name',
 'user_verified',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id_str',
 'hashtags',
 'mentions',
 'urls',
 'media']

In [ ]:
df_spark_subset = df_spark.select('created_at','id_str', 'user_verified', 'favorite_count', 'retweet_count', 'full_text', 'user_id_str')

In [ ]:
df_spark_subset.show()

+--------------------+-------------------+-------------+--------------+-------------+--------------------+-------------------+
|          created_at|             id_str|user_verified|favorite_count|retweet_count|           full_text|        user_id_str|
+--------------------+-------------------+-------------+--------------+-------------+--------------------+-------------------+
|2020-06-01T22:59:...|1267591722126598144|        False|            19|            0|I just got hired ...|          247611935|
|2020-06-01T22:59:...|1267591711905026048|        False|             0|            0|tesla [mergrim mi...|          160589441|
|2020-06-01T22:58:...|1267591424628973574|        False|             0|            0|Tesla - Little Su...|          578148290|
|2020-06-01T22:58:...|1267591344299683841|        False|             0|            0|New post: "Why Ha...|1009977973942964226|
|2020-06-01T22:57:...|1267591151487418372|        False|             0|            0|Meaning if you ha...| 8304

In [ ]:
from pyspark.sql.functions import substring
from pyspark.sql.functions import trim

df_spark_subset = df_spark_subset.withColumn("created_date", substring(trim('created_at'),1,10)) #using substring to get the created date

In [ ]:
df_spark_subset.show()

+--------------------+-------------------+-------------+--------------+-------------+--------------------+-------------------+------------+
|          created_at|             id_str|user_verified|favorite_count|retweet_count|           full_text|        user_id_str|created_date|
+--------------------+-------------------+-------------+--------------+-------------+--------------------+-------------------+------------+
|2020-06-01T22:59:...|1267591722126598144|        False|            19|            0|I just got hired ...|          247611935|  2020-06-01|
|2020-06-01T22:59:...|1267591711905026048|        False|             0|            0|tesla [mergrim mi...|          160589441|  2020-06-01|
|2020-06-01T22:58:...|1267591424628973574|        False|             0|            0|Tesla - Little Su...|          578148290|  2020-06-01|
|2020-06-01T22:58:...|1267591344299683841|        False|             0|            0|New post: "Why Ha...|1009977973942964226|  2020-06-01|
|2020-06-01T22:57:..

In [ ]:
df_spark_subset.select("created_date").distinct().count()

744

#### Dates in the sentiment analysis file are 744, and the dates in the stock price file are 755.

In [ ]:
#Grouping it by created data column to get the no of tweets in each day
tweets_in_a_day = df_spark_subset.groupby('created_date').count()

GROUPING BY CREATED DATA COLUMN TO GET NUMBER OF TWEETS IN EACH DAY

In [ ]:
#Date with the highest number of tweets.
from pyspark.sql.functions import desc
tweets_in_a_day.orderBy(desc("count")).show()

+------------+-----+
|created_date|count|
+------------+-----+
|  2019-11-22|45403|
|  2018-02-07|21168|
|  2018-02-06|17481|
|  2019-11-23|16228|
|  2018-08-08|14061|
|  2018-02-08|13845|
|  2020-09-23|13677|
|  2020-02-04|13507|
|  2018-09-07|13494|
|  2020-08-31|13108|
|  2018-08-07|12783|
|  2018-09-28|12433|
|  2020-09-22|11996|
|  2018-08-17|11525|
|  2020-07-23|11426|
|  2018-09-30|11042|
|  2019-11-24|10973|
|  2018-09-29|10897|
|  2018-05-03|10853|
|  2019-11-25|10583|
+------------+-----+
only showing top 20 rows



CALCULATING THE SUM OF "FAVOURITE_COUNT" AND "RETWEET_COUNT" FOR EACH TWEET 


In [ ]:
from pyspark.sql.functions import expr
df_spark_subset = df_spark_subset.withColumn('sum_tweets_count',expr("favorite_count + retweet_count"))

In [ ]:
df_spark_subset.show(3)

+--------------------+-------------------+-------------+--------------+-------------+--------------------+-----------+------------+----------------+
|          created_at|             id_str|user_verified|favorite_count|retweet_count|           full_text|user_id_str|created_date|sum_tweets_count|
+--------------------+-------------------+-------------+--------------+-------------+--------------------+-----------+------------+----------------+
|2020-06-01T22:59:...|1267591722126598144|        False|            19|            0|I just got hired ...|  247611935|  2020-06-01|            19.0|
|2020-06-01T22:59:...|1267591711905026048|        False|             0|            0|tesla [mergrim mi...|  160589441|  2020-06-01|             0.0|
|2020-06-01T22:58:...|1267591424628973574|        False|             0|            0|Tesla - Little Su...|  578148290|  2020-06-01|             0.0|
+--------------------+-------------------+-------------+--------------+-------------+--------------------+

TEXT CONTENT COLLECTED FOR THE HIGHEST NUMBER OF TWEETS ON JAN 03

In [ ]:
data_top100 = df_spark_subset.filter(trim(df_spark_subset.created_date) =='2019-11-22').select('created_date','full_text','sum_tweets_count').orderBy(desc("sum_tweets_count")).limit(100)

In [ ]:
data_top100.show(30,False)

+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|created_date|full_text                                                                                                                                                                                                                                                                              |sum_tweets_count|
+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|2019-11-22  |YOO Elon Musk was talking about how durable the ne

Write about this day beacasue there are so many tweets idiot 

Model Y came?

In [ ]:
df_spark_subset.columns

['created_at',
 'id_str',
 'user_verified',
 'favorite_count',
 'retweet_count',
 'full_text',
 'user_id_str',
 'created_date',
 'sum_tweets_count']

In [ ]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lower

#df_top100=df_top100.withColumn("text",regexp_replace('text_', '[^a-zA-Z0-9]+', ''))

df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('full_text', '@[A-Za-z0-9_]+', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('text', '#[A-Za-z0-9_]+', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('text', 'http\S+', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('text', '\[.*?\]', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('text', '[()!?]', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace('text', '[$#,:^.;-]', ''))
df_spark_subset=df_spark_subset.withColumn("text",regexp_replace("text", "\'", ""))

In [ ]:
df_spark_subset.select('full_text').show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|full_text                                                                                                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|I just got hired at Tesla! God is good.                                                                                                                                                                                         |
|tesla [mergrim mix] (Remastered 2020) - Spangle call Lilli line - SCLL [Disc 1] #nowplaying

In [ ]:
df_spark_subset.select('text').show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                        |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|I just got hired at Tesla God is good                                                                                                                                                                                       |
|tesla  Remastered 2020  Spangle call Lilli line  SCLL                                                      

In [ ]:
df_spark_subset.columns

['created_at',
 'id_str',
 'user_verified',
 'favorite_count',
 'retweet_count',
 'full_text',
 'user_id_str',
 'created_date',
 'sum_tweets_count',
 'text']

In [ ]:
df_spark_subset.show(3)

+--------------------+-------------------+-------------+--------------+-------------+--------------------+-----------+------------+----------------+--------------------+
|          created_at|             id_str|user_verified|favorite_count|retweet_count|           full_text|user_id_str|created_date|sum_tweets_count|                text|
+--------------------+-------------------+-------------+--------------+-------------+--------------------+-----------+------------+----------------+--------------------+
|2020-06-01T22:59:...|1267591722126598144|        False|            19|            0|I just got hired ...|  247611935|  2020-06-01|            19.0|I just got hired ...|
|2020-06-01T22:59:...|1267591711905026048|        False|             0|            0|tesla [mergrim mi...|  160589441|  2020-06-01|             0.0|tesla  Remastered...|
|2020-06-01T22:58:...|1267591424628973574|        False|             0|            0|Tesla - Little Su...|  578148290|  2020-06-01|             0.0|Te

In [ ]:
columns_to_drop = ['created_at', 'id_str', 'user_verified', 'favorite_count', 'retweet_count', 'full_text', 'sum_tweets_count']
df_spark_subset = df_spark_subset.drop(*columns_to_drop)

In [ ]:
df_spark_subset.columns

['user_id_str', 'created_date', 'text']

In [ ]:
df_spark_subset.show(3)

+-----------+------------+--------------------+
|user_id_str|created_date|                text|
+-----------+------------+--------------------+
|  247611935|  2020-06-01|I just got hired ...|
|  160589441|  2020-06-01|tesla  Remastered...|
|  578148290|  2020-06-01|Tesla  Little Suz...|
+-----------+------------+--------------------+
only showing top 3 rows



In [ ]:
df2 = df_spark_subset.alias('df2')

In [ ]:
df2.show()

+-------------------+------------+--------------------+
|        user_id_str|created_date|                text|
+-------------------+------------+--------------------+
|          247611935|  2020-06-01|I just got hired ...|
|          160589441|  2020-06-01|tesla  Remastered...|
|          578148290|  2020-06-01|Tesla  Little Suz...|
|1009977973942964226|  2020-06-01|New post "Why Has...|
| 830407971565867008|  2020-06-01|Meaning if you ha...|
|1066400744608284672|  2020-06-01|Stocks Moving Pre...|
|         2607358237|  2020-06-01|Why Has Tesla Hal...|
|          535314446|  2020-06-01|Tesla TSLA soars ...|
|          548832414|  2020-06-01|My grandma called...|
|         3257184469|  2020-06-01|One day I will ow...|
|1106659289693745153|  2020-06-01|For anyone doubti...|
|1094850403672281088|  2020-06-01|Life is good for ...|
|1265428606265720833|  2020-06-01|TSLA massive prem...|
|1255159259165126656|  2020-06-01| based coin in th...|
| 806175180682838016|  2020-06-01|Tesla TSLA soa

In [ ]:
df2.columns

['user_id_str', 'created_date', 'text']

In [ ]:
df_spark_subset.show()

+-------------------+------------+--------------------+
|        user_id_str|created_date|                text|
+-------------------+------------+--------------------+
|          247611935|  2020-06-01|I just got hired ...|
|          160589441|  2020-06-01|tesla  Remastered...|
|          578148290|  2020-06-01|Tesla  Little Suz...|
|1009977973942964226|  2020-06-01|New post "Why Has...|
| 830407971565867008|  2020-06-01|Meaning if you ha...|
|1066400744608284672|  2020-06-01|Stocks Moving Pre...|
|         2607358237|  2020-06-01|Why Has Tesla Hal...|
|          535314446|  2020-06-01|Tesla TSLA soars ...|
|          548832414|  2020-06-01|My grandma called...|
|         3257184469|  2020-06-01|One day I will ow...|
|1106659289693745153|  2020-06-01|For anyone doubti...|
|1094850403672281088|  2020-06-01|Life is good for ...|
|1265428606265720833|  2020-06-01|TSLA massive prem...|
|1255159259165126656|  2020-06-01| based coin in th...|
| 806175180682838016|  2020-06-01|Tesla TSLA soa

In [ ]:
# Convert spark into csv file

In [ ]:
df2.write.csv('/content/drive/Shareddrives/IDS561-BigData/Data/SentimentAnalysis/final_experiment/data_cleaning', header=True)